<a href="https://colab.research.google.com/github/Aaditya-Prasad/APML/blob/main/Transformers/APGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# testing github - colab 